In [2]:
import pandas as pd
import sqlite3
import numpy as np

In [3]:
'''
this file is used to check the graph we created using sql, we create another graph with the same concpet 
but by iterating through each row. This is done on a server with lot of memory and is the bruteforce approach
to making the graph. So we do this only for 1000 entries.
'''

conn = sqlite3.connect('Extracted.db')
dfe = pd.read_sql(sql = 'SELECT * FROM Extracted', con = conn)
print(dfe.head())


   Filename     Lemma  CNG        Word  Index1  Index2
0         1    pañcan   41   pañcan_41       0       0
1         1     ratna   41    ratna_41       1       0
2         1    mukhya   41   mukhya_41       2       0
3         1        ca    2        ca_2       3       0
4         1  uparatna    3  uparatna_3       3       1


In [5]:
#This process will be extremly slow as we are iterating through all the vales, use this code to cross check the result obtained from using sql

z=0
global z
df = pd.DataFrame(columns = ['S_no','Entity_1', 'Type_1', 'Entity_2', 'Type_2', 'Weight'])
global df
#df is our graph that we want to create

print(df)


def extraction(start, end):
    
    try:
        for sentence in dfe.Filename.unique():
            #dfe.Filename.unique() returns all the unique filenames through which we will iterate
            
            #if sentence%100 == 0: 
                #print(sentence, end = ' ')
                #to show progress of execution


            temp = pd.read_sql_query("SELECT Lemma, CNG, Word FROM Extracted WHERE Filename = %d" %(sentence), conn)
            #selecting all Lemma, CNG pairs from a scentence/file.

            for i in range(len(temp.Lemma)):
                #iterating through all the values once

                for j in range(len(temp.Lemma)):
                    #iterating through all the values for the 2nd time so that we can compare

                    if i != j : #condition so that values in same row are not counted

                        #lemma lemma edge
                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.Lemma[i]), str(temp.Lemma[j])))
                        #checking if a edge exists in the graph or it needs to be created

                        if values.empty : 
                            #values.empty return true if dataframe is empty
                            #creating the edge
                            df.loc[z] = [z, temp.Lemma[i], 'Lemma', temp.Lemma[j], 'Lemma', 1]
                            z += 1

                        else:
                            #as the edge already exists we just increase the count
                            df.Weight[values['S_no']] += 1


                        #Lemma CNG Edge
                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.Lemma[i]), str(temp.CNG[j])))
                        if values.empty :
                            df.loc[z] = [z, temp.Lemma[i], 'Lemma', str(temp.CNG[j]), 'CNG', 1] 
                            z += 1

                        else:
                            df.Weight[values['S_no']] += 1

                        #Lemma Word Edge

                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.Lemma[i]), str(temp.Word[j])))
                        if values.empty :
                            df.loc[z] = [z, temp.Lemma[i], 'Lemma', temp.Word[j], 'Word', 1]
                            z += 1

                        else:
                            df.Weight[values['S_no']] += 1

                        #CNG CNG edge
                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.CNG[i]), str(temp.CNG[j])))
                        if values.empty :
                            df.loc[z] = [z, str(temp.CNG[i]), 'CNG', str(temp.CNG[j]), 'CNG', 1] 
                            z += 1

                        else:
                            df.Weight[values['S_no']] += 1

                        #CNG Word edge

                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.CNG[i]), str(temp.Word[j])))
                        if values.empty :
                            df.loc[z] = [z, str(temp.CNG[i]), 'CNG', temp.Word[j], 'Word', 1] 
                            z += 1

                        else:
                            df.Weight[values['S_no']] += 1

                        #Word Word Edge

                        values = df.query("Entity_1 == '%s' & Entity_2 == '%s' " %(str(temp.Word[i]), str(temp.Word[j])))
                        if values.empty :
                            df.loc[z] = [z, temp.Word[i], 'Word', temp.Word[j], 'Word', 1] 
                            z += 1

                        else:
                            df.Weight[values['S_no']] += 1
    except:
        pass
    

    print(df.head())
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS Graph")
    df.to_sql('Graph', conn)
    c.close()
    conn.close()


Empty DataFrame
Columns: [S_no, Entity_1, Type_1, Entity_2, Type_2, Weight]
Index: []


In [6]:
#threading the process to increase speed/ parallel processing
#no of files = 442383
#we will check for 1000 files, and use the sql code from GraphFinal.sql on the same 1000 files
w = 0
y = 1000
itr = 1
for x in range(itr):
    
    t1 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t2 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t3 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t4 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t5 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t6 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t7 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t8 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t9 = threading.Thread(target=extraction, args=(w,w+y))
    w += y
    t10 = threading.Thread(target=extraction, args=(w,w+y))
    w += y

                        
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t5.start()
    t6.start()
    t7.start()
    t8.start()
    t9.start()
    t10.start()

    t1.join()
    t2.join()
    t3.join()
    t4.join()
    t5.join()
    t6.join()
    t7.join()
    t8.join()
    t9.join()
    t10.join()